In [ ]:
#Importando modulos
import keras_core
from tensorflow import keras
import tensorflow as tf

tf.test.gpu_device_name()

In [ ]:
#Definiendo la ruta del contenido que vamos a usar
DATASET_PATH = "D:\\Hacking\\Python\\AI_Learning\\Clasificacion_Imagenes\\PetImages"

--Filtro para eliminar la imagenes que son sean JPEG--

In [ ]:
import tensorflow as tf
import os

#Definiendo una función para filtrar las imagenes y quedarnos solo
#las que sean JPEG
def filter_images():
    #Iniciamos un contador
    deleted_imgs = 0
    #Accediendo a los dos directorios donde se encuentran las imagenes
    for folder_name in ("Cat","Dog"):
        #Para acceder a la ruta de cada carpeta
        folder_path = os.path.join(DATASET_PATH, folder_name)
        #Accediendo a cada imagen de cada directorio
        for image in os.listdir(folder_path):
            #Para acceder a cada imagen de la carpeta
            img_path = os.path.join(folder_path, image)
            try:
                #Abrimos la imagen 
                fobj = open(img_path, "rb")
                #Para comprobar si la imagen esta en formato JPEG
                is_jfif = tf.compat.as_bytes("JFIF") in fobj.peek(10)
            finally:
                #Para cerrar la imagen
                fobj.close()
            #Si no esta en el formato que queremos(JPEG)
            if not is_jfif:
                #Augmentamos el contador
                deleted_imgs += 1
                #Eliminamos la imagen correspondiente
                os.remove(img_path)
    
    #Mostramos por pantalla el numero de imagenes eliminadas
    print(f"Imagenes eliminadas: {deleted_imgs}")

In [ ]:
#Llamamos a la función para que el filtrado de imagenes se ejecute
filter_images()

--Conocinedo el tamaño de las imagenes--

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

#Definiendo el tamaño de la figura donde se mostraran las imagenes
plt.figure(figsize=(10,10))

#Para acceder al directorio Dog
folder_path = os.path.join(DATASET_PATH, "Dog")
#Para recorrer las 9 primeras imagenes
for i, image in enumerate(os.listdir(folder_path)[:9]):
    #Para obtener la ruta completa de la imagen
    img_path = os.path.join(folder_path, image)
    #Leeemos la imagen
    img = mpimg.imread(img_path)
    #Organizando la figura
    #En este caso tendremos 3 filas y 3 columnas
    #subplot(nfilas,ncolumnas,index)
    ax = plt.subplot(3,3,i + 1)
    #Mostramos la imagen
    plt.imshow(img)
    #Le ponemos un titulos de la imagen con el tamaño
    plt.title(f"Tamaño: {img.shape[:2][0]} x {img.shape[:2][1]} pixeles")
    #Para que no ponga ejes en la figura
    plt.axis("off")

#Para mostrar la figura terminada
plt.show()

--Definiendo un tamaño común para las imagenes y obteniendo el subconjunto de entrenamiento--

In [ ]:
#Definiendo un tamaño de imagen 
image_size = (180,180)
#Tamaño de conjunto de imagenes(Tamaño de lote)
batch_size = 128

#Para invocar la funcion de obtencion de datos para el entrenamiento
train_ds = keras.utils.image_dataset_from_directory(
    DATASET_PATH,
    validation_split=0.2, #20% de los datos forman parte del subconjunto de validacion
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)
#Para obtener el numero de lotes
print(f"Numero de lotes de entrenamiento: {len(train_ds)}")

--Comprovacion del correcto tamaño de las imagenes--

In [ ]:
#Definiendo el tamaño de la figura donde se mostraran las imagenes
plt.figure(figsize=(10,10))

#Para recorrer nuestro conjunto de datos de entrenamiento, concretamente 1 lote(128 ejemplos)
for images, labels in train_ds.take(1):
    #Para recorrer 9 imagenes
    for i in range(9):
        #Organizando la figura
        #En este caso tendremos 3 filas y 3 columnas
        #subplot(nfilas,ncolumnas,index)
        ax = plt.subplot(3,3,i + 1)
        #Transormando las imagenes a un formato adecuado para que se puedan representar
        plt.imshow(images[i].numpy().astype("uint8"))
        #Le ponemos un titulos de la imagen con el tamaño
        plt.title(f"Tamaño: {images[i].shape[0]} x {images[i].shape[1]} pixeles")
        #Para que no ponga ejes en la figura
        plt.axis("off")

#Para mostrar la figura terminada
plt.show()

--Obtenemos el subconjunto de validacion y pruebas--

In [ ]:
#Para invocar la funcion de obtencion de datos para el entrenamiento
temp_val_ds = keras.utils.image_dataset_from_directory(
    DATASET_PATH,
    validation_split=0.2, #20% de los datos forman parte del subconjunto de validacion
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)
#Para mostrar el numero de lotes de validacion
print(f"Numero de lotes de validación: {len(temp_val_ds)}")

In [ ]:
#De este conjunto de lotes nos quedaremos con la mitad para validacion
#y la otra mitad para pruebas
val_size = int(0.5 * len(temp_val_ds))
val_ds = temp_val_ds.take(val_size)
test_ds = temp_val_ds.skip(val_size)
print(f"La cantidad de lotes para la validación són: {len(val_ds)}")
print(f"La cantidad de lotes para las pruebas són: {len(test_ds)}")

--Otra forma de obtenencion del subconjunto de validacion y pruebas--

In [ ]:
# train_test_split no puede trabajar con objetos Dataset de Tensorflow
# Esto supone un incremento del consumo de memoria RAM
#Por eso lo convertiremos en una lista
val_ds_sk = list(temp_val_ds)

In [ ]:
#Para crear el modelo
from sklearn.model_selection import train_test_split

#Para dividir del conjunto de validación en validación y pruebas
val_ds_sk, test_ds_sk = train_test_split(
    val_ds_sk,
    test_size=0.5,  #Porcentaje para prueba
    random_state=42,    #Semilla para reproducibilidad
)
print(f"La cantidad de lotes para la validación són: {len(val_ds_sk)}")
print(f"La cantidad de lotes para las pruebas són: {len(test_ds_sk)}")

--Definiendo La arquitectura de nuestra red neurona--

In [ ]:
from keras import layers

#Definiendo la dimension de los datos de entrada (pixles x pixeles, rgb)
input_shape = (180,180,3)

#Definiendo la red neuronal, en este caso sera sequencial
fcnn_model = keras.Sequential()

#Definiendo las diferentes capas
#Entrada de la red neuronal
fcnn_model.add(layers.Input(shape=input_shape))

#Escalando las imagenes
fcnn_model.add(layers.Rescaling(1.0 / 255))

#Estirar o aplanar las imagenes para la primera capa densa
fcnn_model.add(layers.Flatten())

#Capa 1 (Numero de neuronas, función matemàtica)
fcnn_model.add(layers.Dense(384, activation='relu'))

#Capa 2 (Numero de neuronas, función matemàtica)
fcnn_model.add(layers.Dense(256, activation='relu'))

#Capa 3 (Numero de neuronas, función matemàtica)
fcnn_model.add(layers.Dense(128, activation='relu'))

#Capa 4 - Output Layer. Terminamos con una neurona
#ya que lo que queremos es una clasificación binaria de los datos(gato o perro)
#(Numero de neuronas, función matemàtica)
#Si fuesen dos o mas neuronas/clases deberiamos cambiar
#la función de activación por 'softmax'
fcnn_model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
#Para ver un resumen de lo que hemos hecho previamente
fcnn_model.summary()

In [ ]:
#Para ver las diferentes capas
fcnn_model.layers

--Configurando nuestra red neuronal--

In [ ]:
#Compilamos el primer modelo de FCNN
#loss --> Classificacion de Error binaria. Si no fuese binaria habria que poner 'categorical_crossentropy'
#optimizer --> Adam de las más utilizadas. Adam(Learning Rate)
#metrics --> Función utilizada para la red neuronal
fcnn_model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(1e-3), metrics=['accuracy'])

--Entrenando nuestra red neuronal--

In [ ]:
#Proceso de entrenamiento
#epochs --> Las vueltas que da sobre los datos de entrenamiento
history = fcnn_model.fit(train_ds, epochs=10, validation_data=val_ds)

--Grafica para ver la tendencia de los valores de error y exactitud--

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

#Transformar la variable de history(donde estan almacenados los datos)
#en un DataFrame para mostrarlos por pantalla
pd.DataFrame(history.history).plot(figsize=(10, 7))
plt.grid(True)
plt.gca().set_ylim(0, 1.2)
plt.xlabel("epochs")
plt.show()

#Si el entrenamiento es adecuado deberiamos ver
#como la linea de fallo tiende a disminuir y
#la linia de exactitud tiende a incrementar

--Para guardar/almacenar el modelo entrenado en el disco y así no perder el tiempo entrenado--

In [ ]:
#Importamos el módulo de guardado
from keras.saving import load_model

#Guardamos el modelo en disco
fcnn_model.save("D:\\Hacking\\Python\\AI_Learning\\Clasificacion_Imagenes.keras")

--Para volver a cargar el modelo ya entrenado--

In [ ]:
#Importamos el módulo de guardado
from keras.saving import load_model

#Para cargar el modelo ya entrenado
fcnn_model_disk = keras.models.load_model("D:\\Hacking\\Python\\AI_Learning\\Clasificacion_Imagenes.keras")

--Predicción de nuevos ejemplos: Para saber como se comporta el modelo frente a nuevos ejemplos--

In [ ]:
#Para evaluar el modelo con el conjunto de datos de pruebas
evaluation_result = fcnn_model_disk.evaluate(test_ds)

#Para mostrar las metrica de evaluación de los ejemplos(precisión y perdida) 
print("Loss:", evaluation_result[0])
print("Accuracy:", evaluation_result[1])

#Los resultados tendrian que oscilar entre los resultados finales del entrenamiento
#para saber que ha habido un correcto funcionamiento del modelo

--Demostración de como actua nuesto modelo entrenado--

In [ ]:
plt.figure(figsize=(10,10))

for images, labels in test_ds.take(1): #take(1) obtiene un lote del conjunto de datos (128 ejemplos)
    for i in range(9):
        ax = plt.subplot(3,3,i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        predictions = fcnn_model_disk.predict(tf.expand_dims(images[i], 0))
        score = float(predictions[0])    
        plt.title(f"Cat: {100 * (1 - score):.2f}%, Dog: {100 * score:.2f}%")
        plt.axis("off")
        
plt.show()

--Mejorando los resultados obtenidos--
IMPORTANTE: REINICIAR EL ENTORNO
Se necesita reiniciar el entorno ya que se encuentra muy cargado y al hacer mejoras importante en los resultados necesitaremos rendimiento del que ya hemos usado previamente

In [ ]:
#Importamos de vuelta los modulos que usaremos
from tensorflow import keras

#Introduciomos variables importantes
DATASET_PATH = "D:\\Hacking\\Python\\AI_Learning\\Clasificacion_Imagenes\\PetImages"

#Lectura de nuestro Train Dataset
image_size = (180,180)
batch_size = 128
train_ds = keras.utils.image_dataset_from_directory(
    DATASET_PATH,
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

#Obtención de nuestro Validation Dataset
temp_val_ds = keras.utils.image_dataset_from_directory(
    DATASET_PATH,
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

#Obtención de nuestro Test Dataset
val_size = int(0.5 * len(temp_val_ds))
val_ds = temp_val_ds.take(val_size)
test_ds = temp_val_ds.skip(val_size)




--Augmentando el conjunto de datos--
Este principio usa las imagenes ya utilizadas por el modelo modificandolas(ya sea cambiando su orientación o rotación)

In [ ]:
#Importando modulo necesario
from keras import layers

#Para transformar las imagenes(horizontal y con una rotación)
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
    ]
)

In [ ]:
#Recorriendo las imagenes del primer batch de entrenamiento transformada
import matplotlib.pyplot as plt

plt.figure(figsize=(10,10))

for images, _ in train_ds.take(1):
    for i in range(9):
        augmented_images = data_augmentation(images)
        ax = plt.subplot(3,3,i + 1)
        plt.imshow(augmented_images[0].numpy().astype("uint8"))
        plt.axis("off")

In [ ]:
import tensorflow as tf

#Aplicamos nuestro "data augmentation" al conjunto de datos de entrenamiento
train_ds = train_ds.map(
    lambda img, label: (data_augmentation(img), label),
    num_parallel_calls=tf.data.AUTOTUNE,
)

--Mejorando la Red Neuronal Artificial--

In [ ]:
#Definiendo el nuevo modelo RNA(API Funcional)
def make_model(input_shape, num_classes):
    
    #Entrada de nuestros datos
    inputs = keras.Input(shape=input_shape)
    
    #El output de la primera capa se junta con el imput de la segunda capa
    #Capa de escalado de imagenes
    x = layers.Rescaling(1.0 / 255)(inputs)
    #Capa convolucional
    x = layers.Conv2D(128, 3, strides=2, padding="same")(x)
    #Capa de normalización
    x = layers.BatchNormalization()(x)
    #Capa de activacion
    x = layers.Activation("relu")(x)
    
    #Guardando la variable con las capas seteadas
    previous_block_activation = x
    
    #Usando un bucle para definir las siguentes capas con diferentes tamaños de neuronas
    for size in [256, 512, 728]:
        #Capa de activación 
        x = layers.Activation("relu")(x)
        #Capa convolucional
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        #Capa de normalización
        x = layers.BatchNormalization()(x)
        
        #Capa de activacion
        x = layers.Activation("relu")(x)
        #Capa convolucional
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        #Capa de normalización
        x = layers.BatchNormalization()(x)
        
        #Capa de agrupación
        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)
        
        #Conexion residual
        residual = layers.Conv2D(size, 1 ,strides=2, padding="same")(previous_block_activation)        
        x = layers.add([x, residual]) #Agregando residuo
        previous_block_activation = x #Reservando el siguiente residuo
    
    #Capa convolucional  
    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    #Capa de normalización
    x = layers.BatchNormalization()(x)
    #Capa de activacion
    x = layers.Activation("relu")(x)
    
    x = layers.GlobalAveragePooling2D()(x)
    #Condicional para la configuración del modelo
    #activation --> la funcion de activación que debe usar
    #units --> Output layer, el numero de neuronas de la ultima cap 
    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes
    
    #Capa de regularización
    x = layers.Dropout(0.5)(x)
    #Output layers o ultima capa
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)

In [ ]:
#Para construir el modelo
#num_classes --> clasificacion binaria en este caso
xception = make_model(input_shape=(180,180,3), num_classes=2)

In [ ]:
#Mostrando resumen
xception.summary()

--Configuración y entrenamiento de la Red Neuronal Artificial--

In [ ]:
#Compilamos el modelo
xception.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(1e-3), metrics=['accuracy'])

In [ ]:
#Variable de entrenamiento del modelo
history = xception.fit(train_ds, epochs=10, validation_data=val_ds)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

pd.DataFrame(history.history).plot(figsize=(10, 7))
plt.grid(True)
plt.gca().set_ylim(0, 1.2)
plt.xlabel("epochs")
plt.show